# ZCTA Proxy for Lackawanna County, PA

This is an adaptation of the Dept. of Treasury [Guidelines for fact-specific proxies](https://home.treasury.gov/policy-issues/coronavirus/assistance-for-state-local-and-tribal-governments/emergency-rental-assistance-program/service-design/fact-specific-proxies) using ZIP Code Tabulation Areas for Lackawanna County, PA. Because the main [Data Pipeline for Emergency Rental Assistance](https://github.com/usds/erap/tree/main/arp_pipeline) is tract/address-based, this is a one-off. However, it follows the same basic conceptual outlines as the main pipeline, and so should be possible to integrate as an additonal output for states and counties if there's greater demand.

The steps are as follows:

- Lookup PA's FIPS code and Lackawanna County's code as well
- Grab the census variables we use in the analysis (median income, median renter income) and do a little cleaning on the census data
- Use the [Census ZCTA-County mapping](https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_county_rel_10.txt) to figure out which ZCTAs are in Lackawanna County, PA, and filter down the census data to just those ZCTAs
- Add in the HUD income limit for the county
- Backfill any missing renter incomes with median incomes for the ZCTA
- Do a comparison of the income-data we have with the income limit to determine our facts for each ZCTA
- Export

In [1]:
import censusdata
import numpy as np
import pandas as pd
from scipy import stats
import enum

In [2]:
%load_ext sql

In [3]:
state_geos = censusdata.geographies(
    censusdata.censusgeo([("state", "*")]), "acs5", 2019
)


In [4]:
statefp = [val for name, val in state_geos.items() if name=='Pennsylvania'][0].params()[0][-1]

In [5]:
statefp

'42'

In [6]:
%%sql postgresql://postgres:postgres@db/geocoder
select * from county where statefp = '42' and name = 'Lackawanna'

1 rows affected.


gid,statefp,countyfp,countyns,cntyidfp,name,namelsad,lsad,classfp,mtfcc,csafp,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,the_geom
1223,42,069,01213675,42069,Lackawanna,Lackawanna County,06,H1,G4020,None,42540,None,A,1188213226,15900575.0,+41.4402841,-075.6096661,...


In [7]:
county_result = _

In [8]:
countyfp = county_result.DataFrame().iloc[0].countyfp

In [9]:
class CensusVars(str, enum.Enum):
    median_renter_income = 'B25119_003E'
    median_income = 'B19013_001E'
    households_count = 'B07013_001E'
    families_count = 'B17026_001E'
    homeowners_count = 'B07013_002E'
    renters_count = 'B07013_003E'

In [10]:
state_data = censusdata.download(
        "acs5",
        2019,
        censusdata.censusgeo([("state", statefp), ("zip code tabulation area", "*")]),
        [val.value for val in CensusVars],
)


In [11]:
NULL_SIGNALS = [-666666666, -333333333, -222222222]
state_data.replace(to_replace=NULL_SIGNALS, value=np.nan, inplace=True)
state_data.columns = [val.name for val in CensusVars]

In [12]:
zcta_county_map = pd.read_csv('zcta_county_rel_10.txt', dtype='str')

In [13]:
county_zctas = set(zcta_county_map[(zcta_county_map.COUNTY==countyfp) & (zcta_county_map.STATE==statefp)].ZCTA5)

In [14]:
state_data['zcta'] = state_data.index.map(lambda idx: idx.params()[-1][-1])

In [15]:
zip_data = state_data[state_data.zcta.isin(county_zctas)].copy()

In [16]:
zip_data.set_index('zcta', inplace=True)

In [17]:
hud = %sql select * from hud.income_limits where state_alpha = 'PA' and county_name ilike 'Lackawanna County'

 * postgresql://postgres:***@db/geocoder
1 rows affected.


In [18]:
hud = hud.DataFrame()

In [19]:
hud.iloc[0].l80_1

39550

In [20]:
zip_data['hud_income_limit'] = hud.iloc[0].l80_1

In [21]:
zip_data['comparison_income'] = zip_data.median_renter_income
zip_data.loc[zip_data.median_renter_income.isna(), 'comparison_income'] = zip_data.median_income

In [22]:
zip_data['does_income_qualify'] = 'False'
zip_data.loc[(~zip_data.comparison_income.isna()) & (zip_data.comparison_income < zip_data.hud_income_limit), 'does_income_qualify'] = 'True'

In [23]:
zip_data

,median_renter_income,median_income,households_count,families_count,homeowners_count,renters_count,hud_income_limit,comparison_income,does_income_qualify
zcta,,,,,,,,,
18424,33500.0,64111.0,5038.0,1459.0,4513.0,525.0,39550,33500.0,True
18434,31066.0,51450.0,3786.0,1051.0,2784.0,1002.0,39550,31066.0,True
18436,38922.0,63047.0,12366.0,3475.0,10262.0,2104.0,39550,38922.0,True
18419,39205.0,56944.0,4326.0,1223.0,3582.0,744.0,39550,39205.0,True
18510,26257.0,34415.0,10841.0,2387.0,4220.0,6621.0,39550,26257.0,True
18518,43469.0,57563.0,7844.0,2095.0,4968.0,2876.0,39550,43469.0,False
18471,NaN,133542.0,655.0,197.0,655.0,0.0,39550,133542.0,False
18653,NaN,83333.0,211.0,42.0,211.0,0.0,39550,83333.0,False
18411,55537.0,80340.0,21005.0,5956.0,18012.0,2993.0,39550,55537.0,False


In [24]:
zip_data[['does_income_qualify', 'hud_income_limit', 'median_renter_income', 'median_income']].to_excel('lackawanna-county-pa.xlsx')